In [4]:
from copy import deepcopy

In [26]:
class StorageObjectLAMMPS:
    def __init__(self):

        """
        Object to restore the LAMMPS defaults

        Details
        -------
        active_dumps : dict
            empty dict
        active_fixes : dict
            empty dict
        integrator : None or str
            None
        thermostat : None or str
            None
        barostat : None or str
            None
        nsteps : None or str
            None
        LAMMPS-Defaults
        ---------------
        run_style='verlet'
        """
        self.active_fixes=dict()  #
        self.active_dumps=dict()  #
        
        self.integrator=None
        self.barostat=None
        self.thermostat=None

        self.nsteps = None 

        self.run_style='verlet'
    def __str__(self):
        STR="Run:\n"
        STR+=" nsteps     : {}\n".format(self.nsteps)
        STR+=" integrator : {}\n".format(self.integrator)
        STR+="   barostat : {}\n".format(self.barostat)
        STR+=" thermostat : {}\n".format(self.thermostat)
        return STR
    
    @property
    def run_properties(self):
        """run_property"""
        prop = dict()
        ##  run type
        # no time integration
        if self.integrator is None:
            prop['type'] = 'none'
        # energy minization
        elif self.integrator == 'minimize':
            prop['type'] = 'EM'
        # some kind of run
        else:
            prop['nsteps'] = self.nsteps
            if self.thermostat is None:
                # NVE (no thermostat & no barostat)
                if self.barostat is None:
                    if self.integrator == 'nve':
                        prop['type'] = 'NVE'
                    else:
                        raise ImplementationError("Something is off.")
                # NPH (no thermostat but a barostat)
                else:
                    prop['type'] = 'NPH'
                    prop['p_start'] = self.press_start
                    prop['p_stop'] = self.press_stop
                    prop['p_rel'] = self.press_relax
                    prop['p_mode']=  self.press_mode
            else:
                # NVT
                if self.barostat is None:
                    prop['type'] = 'NVT'
                    prop['T_start'] = self.temp_start
                    prop['T_stop'] = self.temp_stop
                    prop['T_rel'] = self.temp_relax
                # NPT
                else:
                    prop['type'] = 'NPT'
                    prop['T_start'] = self.temp_start
                    prop['T_stop'] = self.temp_stop
                    prop['T_rel'] = self.temp_relax
                    # barostat
                    prop['p_start'] = self.press_start
                    prop['p_stop'] = self.press_stop
                    prop['p_rel'] = self.press_relax
                    prop['p_mode']=  self.press_mode
        return prop
    
    #def _get_thermostat_info=
    def get_run(self):
        """
        Function returns the run_information
        """

In [27]:
class LogFileReader:
    def __init__(self,filename):
        """
        
        Attributes
        ----------
        runs :
            List of the run objects
        
        Todo
        ----
        rigid bodies
            not implemented ridig body integrator
        multiple integrators / thermostat / barostat
            not implemented yet (so we can temp different groups)
        """
        self.filename = filename
        self._run = StorageObjectLAMMPS()
        
        # list of WARNINGS
        self.WARNINGS = [] 
        # some defaults
        self.runs=[]
        # parse logfile
        self.parse_file(self.filename) 
        
    def parse_file(self,filename):
        """
        Parse a lammps logfile
        """
        with open(filename) as fp:
            for line in fp:
                line_strip=line.strip()
                if len(line_strip) == 0: continue  # empty line
                if line_strip[0] == "#": continue  # comment line
                line_split = line_strip.split()
                if line_strip.find("$") == -1:
                    self.interprete_line(line_strip)
        self.compile_infos()
        
    def interprete_line(self,line):
        """
        Function to interprete the line
        """
        line_split = line.split()
        keyword = line_split[0]
        # Handle fixes
        if   keyword in ['fix', 'unfix']:
            self.handle_fix(line_split)
        elif keyword == 'minimize': 
            self.handle_minimize(line_split)
        elif keyword == 'run':
            self.handle_run(line_split)
        elif keyword == 'WARNING:':
            self.WARNINGS.append(line)
        
    def handle_fix(self,line_split):
        """
        Handle a fix & unfix
        """
        keyword  = line_split[0]
        if   keyword == 'fix':
            fix_id   = line_split[1]  # id
            fix_grp  = line_split[2]  # group
            fix_type = line_split[3]  # type
            fix_args = line_split[4:] # args 
            self._run.active_fixes[fix_id]=[fix_grp,fix_type,fix_args]
        elif keyword == 'unfix':
            del self._run.active_fixes[line_split[1]]
        
    def handle_run(self,line_split):
        """
        Add a run
        """
        
        self.compile_infos()
        
        run = StorageObjectLAMMPS()
        for key in vars(self._run).keys():
            if not key.startswith("_"):
                setattr(run,key,getattr(self._run,key))
        run.nsteps = int(line_split[1])
        if self._run.nsteps is None:
            self._run.nsteps = 0
        self._run.nsteps+= int(line_split[1])
        self.runs.append(run)
        
    def handle_minimize(self,line_split):
        """
        Add a minimization
        """
        
        run = StorageObjectLAMMPS()
        run.integrator = 'minimize'
        run.minimize_prop=dict(
            etol=float(line_split[1]),
            ftol=float(line_split[2]),
            maxiter=int(line_split[3]),
            maxeval=int(line_split[4]),
        )
        self.runs.append(run)
        
    def compile_infos(self):
        """
        Function to compile the infos.
        Finds integrator, thermostat, barostat
        """
        self._run.integrator = None
        self._run.thermostat = None
        self._run.barostat = None
        
        # go over all fix IDS
        for fixid,(fix_grp, fix_type, fix_args) in self._run.active_fixes.iteritems():
            # check if have set an integrator
            if fix_type in ['nve', 'nvt', 'npt', 'nph']:
                assert self._run.integrator is None, "Multiple Integreators are not implemented"
                self._run.integrator = fix_type
                self._run._integrator_id = fixid
            # check for thermostat
            for part in fix_type.split("/"):  # get rid of combinations
                if part in ['nvt', 'npt', 'temp', 'langevin']:
                    assert self._run.thermostat is None, "Multiple Thermostats are not implemented"
                    self._run.thermostat = fix_type
                    self._run._thermostat_id = fixid
                    if part in ['nvt', 'npt']:
                        i = fix_args.index("temp")
                        self._run.temp_start = fix_args[i+1]
                        self._run.temp_stop = fix_args[i+2]
                        self._run.temp_relax = fix_args[i+3]
                        # tchain, tloop and drag, 
                        
                    else:
                        self._run.temp_start = fix_args[0]
                        self._run.temp_stop = fix_args[1]
                        self._run.temp_relax = fix_args[2]
            # check for barostat
            for part in fix_type.split("/"):  # get rid of combinations
                if part in ['nph', 'npt', 'press']:
                    assert self._run.barostat is None, "Multiple Barostats are not implemented"
                    self._run.barostat = fix_type
                    self._run._barostat_id = fixid
                    self._run.press_mode = None
                    for i,arg in enumerate(fix_args):
                        # default couplings
                        if arg in ['iso', 'aniso', 'tri']:
                            self._run.press_mode=arg
                            self._run.press_start = fix_args[i+1]
                            self._run.press_stop = fix_args[i+2]
                            self._run.press_relax = fix_args[i+3]
                        # if its an individual coupling of dimensions
                        elif arg in ['x', 'y', 'z', 'xy', 'yz', 'xz']:
                            if self._run.press_mode is None:
                                self._run.press_mode = []
                                self._run.press_start = []
                                self._run.press_stop = []
                                self._run.press_relax = []
                            self._run.press_mode.append(arg)
                            self._run.press_start.append(fix_args[i+1])
                            self._run.press_stop.append(fix_args[i+2])
                            self._run.press_relax.append(fix_args[i+3])
                        # pchain, mtk, ploop, nreset, drag, and dilate
                            
#             iso or aniso or tri values = Pstart Pstop Pdamp
#     Pstart,Pstop = scalar external pressure at start/end of run (pressure units)
#     Pdamp = pressure damping parameter (time units)
#   x or y or z or xy or yz or xz
#     def __del__(self):
#         for run in self.runs:
#             del run

In [28]:
L = LogFileReader("log.nve.lammps")
for run in L.runs:
    print run.run_properties

{'type': 'NVE', 'nsteps': 100}


In [29]:
L = LogFileReader("log.nvt.lammps")
for run in L.runs:
    print run.run_properties

{'T_rel': '1', 'type': 'NVT', 'nsteps': 100, 'T_start': '1', 'T_stop': '1'}


In [30]:
L = LogFileReader("log.nvt_nvt.lammps")
for run in L.runs:
    print run.run_properties

{'T_rel': '1', 'type': 'NVT', 'nsteps': 100, 'T_start': '1', 'T_stop': '1'}
{'T_rel': '1', 'type': 'NVT', 'nsteps': 1000, 'T_start': '1', 'T_stop': '1'}


In [31]:
L = LogFileReader("log.nvt_npt.lammps")
for run in L.runs:
    print run.run_properties

{'T_rel': '1', 'type': 'NVT', 'nsteps': 100, 'T_start': '1', 'T_stop': '1'}
{'p_rel': '10', 'p_start': '1', 'T_start': '1', 'T_rel': '1', 'p_mode': 'iso', 'p_stop': '1', 'type': 'NPT', 'nsteps': 100, 'T_stop': '1'}


In [32]:
L = LogFileReader("log.nvt.sub_vars.lammps")
for run in L.runs:
    print run.run_properties

{'T_rel': '1', 'type': 'NVT', 'nsteps': 100, 'T_start': '1', 'T_stop': '1'}


In [33]:
L = LogFileReader("log.em.lammps")
for run in L.runs:
    print run.run_properties

{'type': 'EM'}


In [34]:
L = LogFileReader("log.0.lammps")
for run in L.runs:
    print run.run_properties

{'type': 'EM'}
{'T_rel': '0.1', 'type': 'NVT', 'nsteps': 10000, 'T_start': '300', 'T_stop': '300'}
{'p_rel': '1', 'p_start': '1.01325', 'T_start': '300', 'T_rel': '0.1', 'p_mode': 'tri', 'p_stop': '1.01325', 'type': 'NPT', 'nsteps': 10000, 'T_stop': '300'}


In [35]:
L = LogFileReader("log.1.lammps")
for run in L.runs:
    print run.run_properties

{'p_rel': '1', 'p_start': '1.01325', 'T_start': '300', 'T_rel': '0.1', 'p_mode': 'tri', 'p_stop': '1.01325', 'type': 'NPT', 'nsteps': 10000000, 'T_stop': '300'}
